In [ ]:
import requests, csv, flickrapi, flickrapi, urllib, os, re, time, urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse


In [2]:
api_key = '32904a120f4259ca8f4477dcb34bcfa7'
api_secret = '67c4ca1d1c696fde'

In [3]:
username = '198521387@N03'  

In [4]:
url = 'https://www.flickr.com/photos/fractracker/albums/with/72157715916543893'  

In [5]:
username2 = 'FracTracker_Alliance'  

In [7]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

In [9]:
user_info = flickr.people.findByUsername(username=username2)
user_id = user_info['user']['id']

In [10]:
user_info

{'user': {'id': '186454571@N08',
  'nsid': '186454571@N08',
  'username': {'_content': 'FracTracker_Alliance'}},
 'stat': 'ok'}

# How many unique images does FTA have?

In [11]:
unique_photo_ids = set()

#get the list of Fracktracker's albums
photosets = flickr.photosets.getList(user_id=user_id)

In [13]:
type(photosets)

dict

In [38]:
# Write the JSON object to the file
with open('photosets.json', 'w') as file:
    json.dump(photosets, file, indent=4)


JSON object successfully written to photosets.json


In [39]:
! jq photosets.json

jq: error: photosets/0 is not defined at <top-level>, line 1:
photosets.json
jq: 1 compile error


In [37]:
#scrape images from photosets and collect unique photo IDs
for photoset in photosets['photosets']['photoset']:
    photoset_id = photoset['id']

    #make list of photos in the photoset
    photos = flickr.photosets.getPhotos(photoset_id=photoset_id)

    #find unique photo IDs
    for photo in photos['photoset']['photo']:
        photo_id = photo['id']
        unique_photo_ids.add(photo_id)

#scrape images from individual uploads and collect unique photo IDs
individual_photos = flickr.people.getPhotos(user_id=user_id, per_page=500)

for photo in individual_photos['photos']['photo']:
    photo_id = photo['id']
    unique_photo_ids.add(photo_id)

total_unique_images = len(unique_photo_ids)

print(f"The FracTracker_Alliance account has {total_unique_images} unique images across all photo albums.")

The FracTracker_Alliance account has 3130 unique images across all photo albums.


# Scrape Into Attribute Table

In [40]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')
attributes = []

page = 1
per_page = 500
total_pages = 1

user_info = flickr.people.findByUsername(username=username2)
user_id = user_info['user']['id']

# Define the URL for the album
url = 'https://www.flickr.com/photos/fractracker/albums/with/72157715916543893'

# Extract the album ID from the URL
album_id_match = re.search(r'/albums/with/(\d+)', url)
if album_id_match:
    target_album_id = album_id_match.group(1)
    albums = flickr.photosets.getList(user_id=user_id, per_page=per_page)
    for album in albums['photosets']['photoset']:
        album_id = album['id']
        if album_id == target_album_id:
            album_title = album['title']['_content']
            break
    else:
        print("Album not found.")
        album_title = ""

    # Loop through the pages and retrieve the attributes
    while page <= total_pages:
        photos = flickr.photosets.getPhotos(photoset_id=album_id, per_page=per_page, page=page)

        # Create photo_info dictionary
        for photo in photos['photoset']['photo']:
            photo_id = photo['id']
            photo_info = flickr.photos.getInfo(photo_id=photo_id)['photo']

            # Extract basic attributes
            title = photo_info['title']['_content']
            description = photo_info['description']['_content']
            url = f"https://www.flickr.com/photos/fractracker/{photo_id}/in/album-{album_id}"

            # Extract latitude and longitude if available
            latitude = None
            longitude = None
            if 'location' in photo_info:
                location_info = photo_info['location']
                if 'latitude' in location_info and 'longitude' in location_info:
                    latitude = location_info['latitude']
                    longitude = location_info['longitude']

            attributes.append({
                'PhotoID': photo_id,
                'Title': title,
                'Description': description,
                'URL': url,
                'Latitude': latitude,
                'Longitude': longitude,
                'AlbumID': album_id,
                'AlbumTitle': album_title
            })

        page += 1
        total_pages = int(photos['photoset']['pages'])

    df = pd.DataFrame(attributes)
    df.to_csv('flickr_attributes7.csv', index=False)

In [41]:
fa6 = pd.read_csv('flickr_attributes7.csv')
fa6.head()

,PhotoID,Title,Description,URL,Latitude,Longitude,AlbumID,AlbumTitle
0,52748254383,TAuch_Infrastructure-Heritage_HazardousWaste_I...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5274...,40.631783,-80.546412,72157715916543893,Appalachian Buildout
1,52748254428,TAuch_Infrastructure-Heritage_HazardousWaste_I...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5274...,40.630638,-80.545881,72157715916543893,Appalachian Buildout
2,52748254463,TAuch_Infrastructure-Heritage_HazardousWaste_I...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5274...,40.631202,-80.545584,72157715916543893,Appalachian Buildout
3,52748172745,TAuch_Infrastructure-Heritage_HazardousWaste_I...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5274...,40.630791,-80.545256,72157715916543893,Appalachian Buildout
4,52748007544,TAuch_Infrastructure-Heritage_HazardousWaste_I...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5274...,40.631625,-80.546289,72157715916543893,Appalachian Buildout


In [42]:
fa6.shape

(1099, 8)

In [43]:
fa6['AlbumTitle'].value_counts()

AlbumTitle
Appalachian Buildout    1099
Name: count, dtype: int64

In [44]:
fa6.URL.value_counts()

URL
https://www.flickr.com/photos/fractracker/53399032407/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52748254383/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52748254428/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52748254463/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52748172745/in/album-72157715916543893    1
                                                                                   ..
https://www.flickr.com/photos/fractracker/52753470124/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52753470079/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52753634955/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52752690712/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/52753719038/in/album-72157715916543893    1
Name: count, Length: 1099, dtype: int64

# Scrape from all albums

In [ ]:
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')
attributes = []

user_info = flickr.people.findByUsername(username=username2)
user_id = user_info['user']['id']

# Retrieve the list of albums for the user
albums = flickr.photosets.getList(user_id=user_id, per_page=per_page)

# Loop through the albums and extract the attributes
for album in albums['photosets']['photoset']:
    album_id = album['id']
    album_title = album['title']['_content']

    # Loop through the pages and retrieve the attributes for each album
    page = 1
    total_pages = 1
    while page <= total_pages:
        photos = flickr.photosets.getPhotos(photoset_id=album_id, per_page=per_page, page=page)

        # Create photo_info dictionary
        for photo in photos['photoset']['photo']:
            photo_id = photo['id']
            photo_info = flickr.photos.getInfo(photo_id=photo_id)['photo']

            # Extract basic attributes
            title = photo_info['title']['_content']
            description = photo_info['description']['_content']
            url = f"https://www.flickr.com/photos/fractracker/{photo_id}/in/album-{album_id}"

            # Extract latitude and longitude if available
            latitude = None
            longitude = None
            if 'location' in photo_info:
                location_info = photo_info['location']
                if 'latitude' in location_info and 'longitude' in location_info:
                    latitude = location_info['latitude']
                    longitude = location_info['longitude']

            attributes.append({
                'PhotoID': photo_id,
                'Title': title,
                'Description': description,
                'URL': url,
                'Latitude': latitude,
                'Longitude': longitude,
                'AlbumID': album_id,
                'AlbumTitle': album_title
            })

        page += 1
        total_pages = int(photos['photoset']['pages'])

df = pd.DataFrame(attributes)
df.to_csv('flickr_attributes_all.csv', index=False)

In [42]:
faa = pd.read_csv('flickr_attributes_all.csv')
faa.head()

,PhotoID,Title,Description,URL,Latitude,Longitude,AlbumID,AlbumTitle
0,51604730510,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868655,-80.833423,72157715916543893,Appalachian Buildout
1,51604505439,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868177,-80.832106,72157715916543893,Appalachian Buildout
2,51603848491,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868111,-80.832081,72157715916543893,Appalachian Buildout
3,51603848526,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.868288,-80.830231,72157715916543893,Appalachian Buildout
4,51604730665,TAuch_Infrastructure-Rover_Pipeline_Repair-Ene...,"Photo citation: Ted Auch, FracTracker Alliance...",https://www.flickr.com/photos/fractracker/5160...,39.864536,-80.832995,72157715916543893,Appalachian Buildout


In [43]:
faa.shape

(14050, 8)

In [44]:
faa.URL.value_counts()

https://www.flickr.com/photos/fractracker/51604730510/in/album-72157715916543893    1
https://www.flickr.com/photos/fractracker/51119564634/in/album-72157713805048161    1
https://www.flickr.com/photos/fractracker/51141169244/in/album-72157713805048161    1
https://www.flickr.com/photos/fractracker/51141497630/in/album-72157713805048161    1
https://www.flickr.com/photos/fractracker/51141497715/in/album-72157713805048161    1
                                                                                   ..
https://www.flickr.com/photos/fractracker/49883783323/in/album-72157713728924116    1
https://www.flickr.com/photos/fractracker/49883783188/in/album-72157713728924116    1
https://www.flickr.com/photos/fractracker/49884307016/in/album-72157713728924116    1
https://www.flickr.com/photos/fractracker/49883782928/in/album-72157713728924116    1
https://www.flickr.com/photos/fractracker/50199989396/in/album-72157716857731186    1
Name: URL, Length: 14050, dtype: int64

In [45]:
faa.AlbumTitle.value_counts()

Appalachian Buildout                                      1076
Pollution, Toxins, & Disposal Concerns                    1018
Ohio                                                       754
Methane & Air and Water Quality Concerns                   647
Pennsylvania                                               629
Drilling & Fracking                                        612
Waste & Wastewater                                         553
Frac Sand Mining                                           441
Refineries & Processing                                    403
Michigan                                                   393
Construction                                               389
Plastics & Petrochemicals                                  370
Freshwater Ecosystems                                      367
Frac Sand Storage & Processing                             362
Pipelines                                                  348
Wisconsin                                              

In [47]:
faa.Description.value_counts()

Photo citation: Ted Auch, FracTracker Alliance, 2019. Aerial support provided by LightHawk.\n\nEach photo label provides this information, explained below: \n<i>Photographer_topic-sitespecific-siteowner-county-state_partneraffiliation_date(version)</i>\n\nPhoto labels provide information about what the image shows and where it was made. The label may describe the type of infrastructure pictured, the environment the photo captures, or the type of operations pictured. For many images, labels also provide site-specific information, including operators and facility names, if it is known by the photographer. \n\nAll photo labels include location information, at the state and county levels, and at township/village levels if it is helpful. Please make use of the geolocation data we provide - especially helpful if you want to see other imagery made nearby! \n\nWe encourage you to reach out to us about any imagery you wish to make use of, so that we can assist you in finding the best snapshots 

In [48]:
faa.Title.value_counts()

TAuch_Transportation-EastPalestine_Derailment_Site-NorfolkSouthern-ColumbianaCounty-OH_February2023    208
TAuch_Infrastructure-Oil_Dock-Valero_Avery-SanPetricioCounty-TX_December2022                           180
TAuch_Infrastructure-LNG_ExportTerminal-Enbridge-NuecesCounty-TX_December2022                          132
TAuch_Infrastructure-FracSand_Mine-SargentSand-MI_MasonCounty_August2021                               130
TAuch_Infrastructure-Refinery_HartmanPark-Valero-HarrisCounty-TX_December2022                           96
                                                                                                      ... 
TAuch_Infrastructure-StClairRiver-SarniaHarbor-PortHuron-MI_Lighthawk_Sept2021                           1
TAuch_Infrastructure-tar-sands-facility-BP-WhitingOilRefinery-Whiting-CookCounty-IN_July2015(1)          1
TAuch_Infrastructure-tar-sands-facility-BP-WhitingOilRefinery-Whiting-CookCounty-IN_July2015(2)          1
TAuch_Infrastructure-tar-sands-facili